In [44]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import re
import json
import csv
from datetime import datetime
import random
import time
import pandas as pd

In [45]:
session = requests.Session()
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36",
           "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"}
url = "https://en.wikipedia.org/wiki/List_of_countries_by_electricity_production_from_renewable_sources"
req = session.get(url, headers=headers)

In [46]:
driver = webdriver.Chrome(executable_path='/usr/local/bin/chromedriver')
driver.get(url)
time.sleep(10)
driver.implicitly_wait(1)
html = driver.page_source
bsObj = BeautifulSoup(html, "html.parser")

In [47]:
table = bsObj.find("table", {"class": "wikitable"}).find("tbody")

In [48]:
listOfCountries = []
for row in bsObj.find("table", {"class": "wikitable"}).find("tbody").findAll('tr'):
    countryName = row.findAll('td')[0].find('a')['title']
    percOfTotal = row.findAll('td')[2].get_text()[:-1]
    totalGenerated = row.findAll('td')[3].get_text()
    totalRenewableGenerated = row.findAll('td')[4].get_text()
    hydropower = row.findAll('td')[5].get_text()
    windpower = row.findAll('td')[6].get_text()
    biomassAndWaste = row.findAll('td')[7].get_text()
    solarpower = row.findAll('td')[8].get_text()
    geothermal = row.findAll('td')[9].get_text()
    countryEntry = {'name': countryName, 'percOfTotal': percOfTotal, 'totalGenerated':totalGenerated,
                   'totalRenewableGenerated':totalRenewableGenerated, 'hydropower':hydropower, 'windpower': windpower,
                   'biomassAndWaste':biomassAndWaste, 'solarpower':solarpower, 'geothermal':geothermal}
    listOfCountries.append(countryEntry)

In [49]:
listOfCountries

[{'name': 'Afghanistan',
  'percOfTotal': '',
  'totalGenerated': '',
  'totalRenewableGenerated': '1070.9',
  'hydropower': '1035.3',
  'windpower': '0.1',
  'biomassAndWaste': '',
  'solarpower': '35.5',
  'geothermal': ''},
 {'name': 'Albania',
  'percOfTotal': '100',
  'totalGenerated': '7,782',
  'totalRenewableGenerated': '7783.9',
  'hydropower': '7782',
  'windpower': '',
  'biomassAndWaste': '',
  'solarpower': '1.9',
  'geothermal': ''},
 {'name': 'Algeria',
  'percOfTotal': '0.6',
  'totalGenerated': '70,997',
  'totalRenewableGenerated': '430.5',
  'hydropower': '72',
  'windpower': '19.4',
  'biomassAndWaste': '',
  'solarpower': '339.1',
  'geothermal': ''},
 {'name': 'Angola',
  'percOfTotal': '70.3',
  'totalGenerated': '10,361',
  'totalRenewableGenerated': '7282.3',
  'hydropower': '7109',
  'windpower': '',
  'biomassAndWaste': '155',
  'solarpower': '18.3',
  'geothermal': ''},
 {'name': 'Anguilla',
  'percOfTotal': '',
  'totalGenerated': '',
  'totalRenewableGener

In [65]:
data = pd.DataFrame(listOfCountries)

In [66]:
with open('custom.geo.json') as json_file:
    json_data = json.load(json_file)

In [67]:
africa_countries = []
for feature in json_data['features']:
    africa_countries.append(feature['properties']['name'])

In [68]:
africa_countries

['Angola',
 'Burundi',
 "Côte d'Ivoire",
 'Benin',
 'Burkina Faso',
 'Botswana',
 'Cameroon',
 'Dem. Rep. Congo',
 'Congo',
 'Comoros',
 'Central African Rep.',
 'Cape Verde',
 'Djibouti',
 'Algeria',
 'Egypt',
 'Eritrea',
 'Ethiopia',
 'Gabon',
 'Ghana',
 'Guinea',
 'Gambia',
 'Guinea-Bissau',
 'Eq. Guinea',
 'Kenya',
 'Liberia',
 'Libya',
 'Madagascar',
 'Morocco',
 'Lesotho',
 'Mali',
 'Mauritania',
 'Malawi',
 'Namibia',
 'Niger',
 'Mozambique',
 'Nigeria',
 'Rwanda',
 'W. Sahara',
 'Sudan',
 'S. Sudan',
 'Senegal',
 'Sierra Leone',
 'Somaliland',
 'Somalia',
 'São Tomé and Principe',
 'Swaziland',
 'Chad',
 'Togo',
 'Tunisia',
 'Tanzania',
 'Uganda',
 'South Africa',
 'Zambia',
 'Zimbabwe']

In [93]:
name_change = {'Ivory Coast':"Côte d'Ivoire",
            'Democratic Republic of the Congo':'Dem. Rep. Congo',
            'Eswatini': 'Swaziland',
            'São Tomé and Príncipe':"São Tomé and Principe",
            'Republic of the Congo':'Congo',
            'Central African Republic':'Central African Rep.',
            'The Gambia':'Gambia',
            'Equatorial Guinea':'Eq. Guinea',
            'South Sudan':'S. Sudan'}
data['name'] = data['name'].replace(name_change)
data['totalGenerated'] = data['totalGenerated'].str.replace(',','')

In [94]:
for index, row in data.iterrows():
    if row['name'] not in africa_countries:
        data.drop(index, inplace=True)

In [95]:
data.to_csv('renewable_energy_africa_data.csv')

In [97]:
len(africa_countries), data.shape

(54, (52, 9))